# Softmax for probabilities - OvO vs. OvR - ALL features

In [1]:
import numpy as np
import matplotlib.pylab as plt

plt.ion()

In [2]:
import sys
sys.path.append('../../data')
from corpus import load_corpus
data = load_corpus('weebit')

X_train = data['X_train']
y_train = data['y_train']
y_train_onehot = data['y_train_onehot']
X_test = data['X_test']
y_test = data['y_test']
y_test_onehot = data['y_test_onehot']

Using TensorFlow backend.


In [3]:
from sklearn.metrics import accuracy_score, make_scorer
from sklearn.model_selection import cross_val_score

def threshold_socre(y_true, y_pred):
    return np.sum(np.abs(y_true - y_pred) <= 1) / len(y_true)

def calc_scores(y_true, y_pred_probs):
    y_pred_avg = (y_pred_probs * np.arange(5)).sum(axis=1)
    y_pred_avg_classes = y_pred_avg.round().clip(0, 4).astype(int)
    return {'accuracy': accuracy_score(y_true, y_pred_avg_classes),
            'threshold': threshold_socre(y_true, y_pred_avg_classes)}

threshold_scorer = make_scorer(threshold_socre)
accuracy_scorer = make_scorer(accuracy_score)

def calc_scores_with_cv(model, X, y, cv=5):
    return {'accuracy': np.mean(cross_val_score(model, X, y, scoring=accuracy_scorer, cv=cv)),
            'threshold': np.mean(cross_val_score(model, X, y, scoring=threshold_scorer, cv=cv))}

## OvO model - First SVM probabilities and then Softmax ones

In [4]:
from sklearn.svm import SVC

model_ovo = SVC(kernel='rbf', C=1, probability=True, decision_function_shape='ovo')

print('CV:', calc_scores_with_cv(model_ovo, X_train, y_train))

model_ovo.fit(X_train, y_train)

y_pred = model_ovo.predict(X_test)
y_pred_probs = model_ovo.predict_proba(X_test)
print('Test:', calc_scores(y_test, y_pred_probs))

CV: {'threshold': 0.9143854547111088, 'accuracy': 0.708029635958036}
Test: {'threshold': 0.9436813186813187, 'accuracy': 0.6442307692307693}


In [5]:
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier


def create_prob_calculator_ovo():
    prob_calculator_ovo = Sequential()
    prob_calculator_ovo.add(Dense(5, input_dim=10, activation='softmax'))

    prob_calculator_ovo.compile(optimizer='rmsprop',
                        loss='categorical_crossentropy',
                        metrics=['accuracy'])

    return prob_calculator_ovo


prob_calculator_ovo = KerasClassifier(build_fn=create_prob_calculator_ovo,
                                      epochs=10, 
                                      verbose=0)


X_dist_ovo_train = model_ovo.decision_function(X_train)
X_dist_ovo_test = model_ovo.decision_function(X_test)

print('CV:', calc_scores_with_cv(prob_calculator_ovo, X_dist_ovo_train, y_train))


prob_calculator_ovo.fit(X_dist_ovo_train, y_train_onehot)

y_pred_probs = prob_calculator_ovo.predict_proba(X_dist_ovo_test)
print('Test:', calc_scores(y_test, y_pred_probs))

CV: {'threshold': 0.9425744213969279, 'accuracy': 0.7895434462444773}
Test: {'threshold': 0.9326923076923077, 'accuracy': 0.6346153846153846}


## OvR model - First SVM probabilities and then Softmax ones

In [6]:
from sklearn.multiclass import OneVsRestClassifier

model_ovr = OneVsRestClassifier(SVC(kernel='rbf', C=1, probability=True))

print('CV:', calc_scores_with_cv(model_ovr, X_train, y_train))

model_ovr.fit(X_train, y_train)

y_pred = model_ovo.predict(X_test)
y_pred_probs = model_ovr.predict_proba(X_test)
print('Test:', calc_scores(y_test, y_pred_probs))

CV: {'threshold': 0.9078503243165768, 'accuracy': 0.7042424757382629}
Test: {'threshold': 0.929945054945055, 'accuracy': 0.5755494505494505}


In [7]:
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier


def create_prob_calculator_ovr():
    prob_calculator_ovr = Sequential()
    prob_calculator_ovr.add(Dense(5, input_dim=5, activation='softmax'))

    prob_calculator_ovr.compile(optimizer='rmsprop',
                        loss='categorical_crossentropy',
                        metrics=['accuracy'])

    return prob_calculator_ovr


prob_calculator_ovr = KerasClassifier(build_fn=create_prob_calculator_ovr,
                                      epochs=10, 
                                      verbose=0)


X_dist_ovr_train = model_ovr.decision_function(X_train)
X_dist_ovr_test = model_ovr.decision_function(X_test)

print('CV:', calc_scores_with_cv(prob_calculator_ovr, X_dist_ovr_train, y_train))


prob_calculator_ovr.fit(X_dist_ovr_train, y_train_onehot)

y_pred_probs = prob_calculator_ovr.predict_proba(X_dist_ovr_test)
print('Test:', calc_scores(y_test, y_pred_probs))

CV: {'threshold': 0.9356731787237316, 'accuracy': 0.8101673261529181}
Test: {'threshold': 0.9326923076923077, 'accuracy': 0.5343406593406593}
